#### Step 0: データの読み込み ##### 

In [1]:
import geopandas as gpd
import pandas as pd

#墨田区の100m四方のメッシュポリゴン(shp)の読み込み
esh = gpd.read_file("data/Sumida_mesh.geojson")

#各メッシュの中心点と各SAP(service access points：徒歩640m圏内にあるバス停、徒歩960m圏内にある駅）を結んだshpの読み込み
mesh_busstop = gpd.read_file("data/mesh_to_busstops.geojson")
mesh_station = gpd.read_file("data/mesh_to_stations.geojson")

#各バス停における各バスルートの運行頻度データ(csv)の読み込み
busstop = pd.read_csv("data/BusStops.csv")
station = pd.read_csv("data/Stations.csv")

In [2]:
mesh_busstop.columns

Index(['OBJECTID', 'Name', 'OriginID', 'DestinationID', 'DestinationRank',
       'Total_WalkTime', 'Total_Kilometers', 'Name_tentative',
       'OriginID_tentative', 'Shape_Length', 'MeshID', 'BusStopID',
       'geometry'],
      dtype='object')

#### Step 1: 各SAPへの歩行時間の計算 ##### 

In [3]:
#メッシュの中心点とSAPを結ぶデータには所要時間も入っている（'Total_Walk'）が、
#5km/hの歩行速度で計算されたものなので、
#距離データ（'Total_Kilo'）を使って、4.8km/hの歩行速度での所要時間に変換

mesh_busstop['WalkTime'] = mesh_busstop['Total_Kilometers']/4.8*60
mesh_station['WalkTime'] = mesh_station['Total_Kilometers']/4.8*60

#### Step 2: 各SAPの予定待ち時間（SWT）の算出 ##### 

In [4]:
#SWT(scheduled waiting time) = 0.5 * (60/頻度)
busstop['SWT'] = 0.5*(60/ busstop['PeakHourFq'])
station['SWT'] = 0.5*(60/ station['PeakHourFq'])

#### Step 3: 各SAPの平均待ち時間（AWT）の算出 ##### 

In [5]:
#AWT(average waiting times)=SWT + 信頼係数
#バスには信頼性係数2分、鉄道、地下鉄、路面電車には信頼性係数0.75分
busstop['AWT'] = busstop['SWT']+2
station['AWT'] = station['SWT']+0.75

#### Step 4: 各SAPの各ルートの合計アクセス時間（TAT）の算出 ##### 

In [10]:
joined_mesh_busstop = pd.merge(mesh_busstop, busstop, on='BusStopID', how='left', suffixes=('', '_from_busstop'))
joined_mesh_station = pd.merge(mesh_station, station, on='StationID', how='left', suffixes=('', '_from_station'))
#joinをする（many to many）：左にgdf、右にcsv、BusStopIDとStationIDをキーに
#joinの前後の行の数を確認
#やってる操作が合っているかどうかの再確認を

In [ ]:
#TAT(total access time) = 移動時間（'WalkTime'）+AWT



#### Step5: 各SAPのEDFの算出 ##### 

In [9]:
#EDF(equivalent doorstep frequency) = 0.5 * (60 / TAT)
#EDFはTATを時間ではなく頻度の単位に変換したもの



#### Step6: アクセス指数（AI）の算出 ##### 

In [ ]:
#MeshIDごとにまとめる



#### Step 7: PTALへの変換 ##### 